In [ ]:
#mount
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

#careful; these were names the same but appear to be different  for whatever reason
#these were modified in the Dataset_CA folder (CA stands for cliinically aquired)
#GEO-PI-003-NMH_MAIN.nii --> same
#GEO-PI-003-NMH-JM.nii --> GEO-PI-0031-NMH-JM.nii
#GEO-PII-004-NMH_MAIN.nii --> same
#GEO-PII-004-NMH-PP.nii --> GEO-PII-0041-NMH-PP.nii


Mounted at /content/gdrive/


In [ ]:
#pip install
!pip install --quiet SimpleITK

     |████████████████████████████████| 52.8 MB 225 kB/s 


In [ ]:
#import libraries
import glob
import os
import pandas as pd
import numpy as np
import shutil
from shutil import copyfile
import SimpleITK as sitk
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
#identify the patients to remove

#specify the path to the CTAs and SegGT (Both Old and New)
path_CTA = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/CTA/'
path_SegGT = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/SegGT/'
hospital = 'NMH'

#get the save paths
path_CTA_new = path_CTA[:-1] + '_New/'
path_CTA_old = path_CTA[:-1] + '_Old/'
path_SegGT_new = path_SegGT[:-1] + '_New/'
path_SegGT_old = path_SegGT[:-1] + '_Old/'

#identify the samples which may cause issues (from the old dataset)
ls = []
path_to_old = sorted(os.listdir(path_CTA_old))
for patient in path_to_old:
  if hospital in patient:
    ls.append(patient.replace('_MAIN.nii.gz', '').replace('-NMH',''))

#patients to remove as per the info on the old set
print(ls)

['GEO-PI-002', 'GEO-PI-003', 'GEO-PII-001', 'GEO-PII-004']


In [ ]:
#resampling
def resample_image_standardize(itk_image, out_size = (64,64,64), is_label = False):
  original_spacing = itk_image.GetSpacing()
  original_size = itk_image.GetSize()
  out_spacing = [original_size[0] * (original_spacing[0] / out_size[0]),
                 original_size[1] * (original_spacing[1] / out_size[1]),
                 original_size[2] * (original_spacing[2] / out_size[2])]

  resample = sitk.ResampleImageFilter()
  resample.SetOutputSpacing(out_spacing)
  resample.SetOutputOrigin(itk_image.GetOrigin())
  resample.SetSize(out_size)
  resample.SetOutputDirection(itk_image.GetDirection())
  resample.SetTransform(sitk.Transform())
  #resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
  if is_label:
      resample.SetInterpolator(sitk.sitkNearestNeighbor)
  else:
      resample.SetInterpolator(sitk.sitkBSpline)
  return resample.Execute(itk_image)

#generate a binary mask
def binarize(lower, upper, image, binary_filter):
  binary_filter.SetLowerThreshold(lower)
  binary_filter.SetUpperThreshold(upper)
  return binary_filter.Execute(image)

#helper functions
def find_files(path, ext = '.nii.gz'):
  ls = []
  for file in sorted(os.listdir(path)):
    if file.endswith(ext):
      ls.append(os.path.join(path, file))
  return ls

def set_frames(path1, path2):
  ls = []
  for CT_path, GT_path in zip(find_files(path1, '.nii.gz'), find_files(path2, '.nii.gz')):
    ls.append([CT_path, GT_path])
  visualize1 = pd.DataFrame(ls, columns = ['Original CTA', 'Original SegGT'])
  return visualize1

#prepare the new data
def prepare(row, path_CTA = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/CTA/', path_SegGT = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/SegGT/'):
  #get name
  path = row['Original CTA']
  #get file
  idx = path.rindex('/')
  #get patient
  name = path[idx+1:]
  #get patient name
  patient = path[idx+1:]
  #get idx
  if 'MAIN' in name:
    idx = patient.rindex('_')
  else:
    idx = patient.rindex('-')
  #get the save_name
  save_name = patient[:idx]
  #get description
  _, status, _ , hospital, = save_name.rsplit('-')
  #save
  if status == 'PI':
    status = 'Elective Repair'
  if status == 'PII':
    status = 'Surveillance'
  #return
  return status, hospital, save_name, path_CTA + save_name + '.nii.gz', path_SegGT + save_name + '.nii.gz'

#export
def export(row):
  CTA = sitk.ReadImage(row['Original CTA'])
  #normalize just in case used in training for TorchIO
  z = sitk.NormalizeImageFilter()
  CTA = z.Execute(CTA)
  sitk.WriteImage(CTA, row['CTA'])
  SegGT = sitk.ReadImage(row['Original SegGT'])
  sitk.WriteImage(SegGT, row['SegGT'])

#crop the data
def cropper(CTA, GT, AAA, out_size):
  #filter
  label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
  #apply
  label_shape_filter.Execute(GT)
  #get bbox
  bbox = label_shape_filter.GetBoundingBox(1) #in pixel coordinates
  #get ROI
  CTA = sitk.RegionOfInterest(CTA, bbox[int(len(bbox)/2):], bbox[0:int(len(bbox)/2)])
  #get ROI
  AAA = sitk.RegionOfInterest(AAA, bbox[int(len(bbox)/2):], bbox[0:int(len(bbox)/2)])
  #standardize
  return resample_image_standardize(CTA, out_size, False), resample_image_standardize(AAA, out_size, True), CTA, AAA

#save image
def save_image(save_path, save_folder, patient, image):
  #save file
  if os.path.isdir(save_path + save_folder) == False:
    os.mkdir(save_path + save_folder)
  #save file
  save_loc = save_path + save_folder + '/' + patient + '.nii.gz'
  #write
  sitk.WriteImage(image, save_loc)
  #return
  return save_loc

#process the images
def process_images(row, save_path = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/', out_size = (64, 64, 64)):
  #CTA
  CTA = sitk.ReadImage(row['CTA'])
  #GT
  GT = sitk.ReadImage(row['SegGT'])
  #create masks of the two groups
  binary_filter = sitk.BinaryThresholdImageFilter()
  #z-norm
  z = sitk.NormalizeImageFilter()
  #resample to 64
  CTA_64 = resample_image_standardize(CTA, out_size, is_label = False)
  #z-norm after resampling
  CTA_64 = z.Execute(CTA_64) #step 1
  #BB-AAA

  #binarize the AAA, ILT, and calcifications
  AAA_ILT_Calc = binarize(2, 4, GT, binary_filter)
  #binarize the AAA
  AAA = binarize(2, 2, GT, binary_filter) #might want to include the thrombus itself? (multiclass?)
  #resample the AAA. ILT, and calcifications to 64
  AAA_ILT_Calc_64 = resample_image_standardize(AAA_ILT_Calc, out_size, is_label = True) #step 1
  #crop to AAA_ILT_Calc for AAA
  crop_CTA_64, crop_AAA_64, crop_CTA, crop_AAA = cropper(CTA, AAA_ILT_Calc, AAA, out_size) #step 2

  #z-norm after resampling
  crop_CTA_64 = z.Execute(crop_CTA_64) #step 1

  #Control (no insertion of prediction)
  AAA_64 = resample_image_standardize(AAA, out_size, is_label = True)

  #save the information
  save_path_AAA_ILT_Calc = save_image(save_path, 'AAA-ILT-Calc', row['Patient'], AAA_ILT_Calc)
  save_path_AAA = save_image(save_path, 'AAA', row['Patient'], AAA)
  save_path_CTA_64 = save_image(save_path, 'CTA-64', row['Patient'], CTA_64)
  save_path_AAA_ILT_Calc_64 = save_image(save_path, 'AAA-ILT-Calc-64', row['Patient'], AAA_ILT_Calc_64)
  save_path_crop_CTA_64 = save_image(save_path, 'crop-CTA-64', row['Patient'], crop_CTA_64)
  save_path_crop_AAA_64 = save_image(save_path, 'crop-AAA-64', row['Patient'], crop_AAA_64)
  save_path_crop_CTA = save_image(save_path, 'crop-CTA', row['Patient'], crop_CTA)
  save_path_AAA_64  = save_image(save_path, 'AAA-64', row['Patient'], AAA_64)

  #return
  return save_path_AAA_ILT_Calc, save_path_AAA, save_path_CTA_64, save_path_AAA_ILT_Calc_64, save_path_crop_CTA_64, save_path_crop_AAA_64, save_path_crop_CTA, save_path_AAA_64

In [ ]:
#get the frames
all_df = set_frames('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/Dataset_CA/CTA/', '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/Dataset_CA/SegGT/')
#all
all_df['Status'], all_df['Hospital'], all_df['Patient'], all_df['CTA'], all_df['SegGT'] = zip(*all_df.apply(prepare, axis = 1))
#combine
df = all_df.reset_index(drop = True)

In [ ]:
if os.path.isdir(path_SegGT) == False:
  os.mkdir(path_SegGT)

In [ ]:
%%time
#export all the data
if os.path.isdir(path_CTA) == False:
  os.mkdir(path_CTA)
if os.path.isdir(path_SegGT) == False:
  os.mkdir(path_SegGT)

#export (z-)
_ = df.apply(export, axis = 1)

#save
df.to_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/dataset.pkl')

CPU times: user 22min 18s, sys: 30.6 s, total: 22min 49s
Wall time: 24min 25s


In [ ]:
#now need to construct the model inputs around the splitting!
#want z-normalization
#masks of ILT, AAA, calcifications  do not need, aorta
#masks of AAA, calcifications, ILT
#masks of AAA only?
#cropped regions of the same
#how many images have all 4?
#chould z-normalize the original CTs for the TorchIO

In [ ]:
#params (should be 54 individual people total)
#df['Patient'].value_counts()

In [ ]:
#read
df = pd.read_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/dataset.pkl')

In [ ]:
%time
#process the images
df['AAA-ILT-Calc'], df['AAA'], df['CTA-64'], df['AAA-ILT-Calc-64'], df['crop-CTA-64'], df['crop-AAA-64'], df['crop-CTA'], df['AAA-64']  = zip(*df.apply(process_images, axis = 1))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


In [ ]:
#save the file
df.to_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/data.pkl')

In [ ]:
#need to split the data
df = pd.read_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/data.pkl')

In [ ]:
#function used to obtain additional data (like 3D ViT Classification + Interpret)
def get_more_data(row, im_type, new_size, is_label, folder_name, save_path = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/'):
  #read
  CTA = sitk.ReadImage(row[im_type])
  #resample
  resamp_CTA = resample_image_standardize(CTA, out_size = new_size, is_label = is_label)
  #normalize
  z = sitk.NormalizeImageFilter()
  resamp_CTA = z.Execute(resamp_CTA)
  #do windowing?
  #save the filepath!
  return save_image(save_path, folder_name, row['Patient'], resamp_CTA)

#do some label encoding based on the two classes
def label_encode(row):
  if row['Status'] =='Elective Repair':
    status = 1
  elif row['Status'] == 'Surveillance':
    status = 0
  return status

#convert timage to model inputs
def image2inputs(row, window, out_size, save_col, z = sitk.NormalizeImageFilter(), save_path = '/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/'):
  #read
  image = sitk.ReadImage(row['CTA'])
  #apply windowing if true
  if window is not None:
    image = window.Execute(image)
  #resample
  image = resample_image_standardize(image, out_size, is_label = False)
  #z normalize
  image = z.Execute(image)
  #save image
  return save_image(save_path, save_col, row['Patient'], image) #include the image type later?

#generate the window
def get_window(win_params):
  #if win params exists
  if win_params is not None:
    #define
    window = sitk.IntensityWindowingImageFilter()
    #set params
    if win_params[0] is not None:
      window.SetWindowMinimum(win_params[0])
    if win_params[1] is not None:
      window.SetWindowMaximum(win_params[1])
    if win_params[2] is not None:
      window.SetOutputMinimum(win_params[2])
    if win_params[3] is not None:
      window.SetOutputMaximum(win_params[3])
  else:
    window = None
  #return
  return window

In [ ]:
#params
win_params = [0, None, 0, 1000]
window = get_window(win_params)
out_size = (256, 256, 256)
save_col = 'CTA-256-256-256'

In [ ]:
%%time
#need to save the data (resample) in the correct format use 256-256-256
df[save_col] = df.apply(image2inputs, axis = 1, args = (window, out_size, save_col))

CPU times: user 17min 14s, sys: 31.4 s, total: 17min 46s
Wall time: 18min 2s


In [ ]:
%%time
#apply will take half an hour might be able to get away with different patch sizes so long as when divided by each dimension get 10, 10, 10
#folder_name = 'CTA-256-256-256' #128-128-128?
#df[folder_name] = df.apply(get_more_data, axis = 1, args = ('CTA', (256, 256, 256), False, folder_name))

CPU times: user 31min, sys: 38.8 s, total: 31min 39s
Wall time: 27min 11s


In [ ]:
#save
df['Label'] = df.apply(label_encode, axis = 1)
df.to_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/data.pkl')

In [ ]:
#do the data splitting (do not change these lines otherwise patient splitting for prior experiments will change!)
df_train, df_test = train_test_split(df, train_size = 0.46, stratify = df[[
    'Status', 'Hospital',
    ]], shuffle = True, random_state = 42)
df_train['DATA'] = 'TRAIN'
df_test['DATA'] = 'TEST'
df = pd.concat([df_train, df_test])
print('Train ', len(df_train))
print('Test ', len(df_test))

Train  24
Test  30


In [ ]:
#save the data splitting
df.to_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/data_split.pkl')

In [ ]:
#can also add functionality to count the volume of the aneurysm, ILT, and calcifications later or dertmine their prevalence and such

In [ ]:
#print out some data #all have 1 and 2 maybe get thrmbus if multiclass
dir = sorted(os.listdir('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/SegGT/'))
for path in dir:
  image = sitk.ReadImage('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/dataset/SegGT/' + path)
  print(np.unique(sitk.GetArrayFromImage(image), return_counts = True))

In [ ]:
#GEO-PI-003-NMH     2
#GEO-PII-004-NMH    2
#these two appear to be different images but are labeled as the same patient (included in the dataset_CA with -1 extension)

In [ ]:
df = pd.read_pickle('/content/gdrive/MyDrive/AAA_Project/Masters-Thesis/AAA-DICOM/data/data_split.pkl')

In [ ]:
for l1, l2 in zip(ls1, ls2):
  if l1 not in l2:
    print(l1)